In [3]:
pip install seaborn

   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/294.9 kB 640.0 kB/s eta 0:00:01
   ------------ --------------------------- 92.2/294.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
print('hello')

hello


In [6]:
'''
 Compiles and filters different data metrics from TrackMate's tracks output file

 Assumes all output files are in different folders within one main directory

 Processes one condition
'''

import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

df_SPOTS = pd.DataFrame()
df_GAPS = pd.DataFrame()
df_SPLITS = pd.DataFrame()
df_MERGES = pd.DataFrame()
df_COMPLEX = pd.DataFrame()
df_LONG_GAP = pd.DataFrame()
df_START = pd.DataFrame()
df_STOP = pd.DataFrame()
df_DISPLACEMENT = pd.DataFrame()
df_DURATION = pd.DataFrame()
df_X_LOC = pd.DataFrame()
df_Y_LOC = pd.DataFrame()
df_Z_LOC = pd.DataFrame()
df_MEAN_SPEED = pd.DataFrame()
df_MAX_SPEED = pd.DataFrame()
df_MIN_SPEED = pd.DataFrame()
df_MEDIAN_SPEED = pd.DataFrame()
df_STD_SPEED = pd.DataFrame()
df_MEAN_QUAL = pd.DataFrame()
df_TOT_DIST = pd.DataFrame()
df_MAX_DIST = pd.DataFrame()
df_CONFINEMENT = pd.DataFrame()
df_MEAN_LINE_SPEED = pd.DataFrame()
df_LINEARITY = pd.DataFrame()
df_MEAN_DIRECTION = pd.DataFrame()

df_list = [df_SPOTS, df_GAPS, df_SPLITS, df_MERGES, df_COMPLEX, df_LONG_GAP, df_START, df_STOP,
           df_DISPLACEMENT, df_DURATION, df_X_LOC, df_Y_LOC, df_Z_LOC, df_MEAN_SPEED,
           df_MAX_SPEED, df_MIN_SPEED, df_MEDIAN_SPEED, df_STD_SPEED, df_MEAN_QUAL,
           df_TOT_DIST, df_MAX_DIST, df_CONFINEMENT, df_MEAN_LINE_SPEED, df_LINEARITY,
           df_MEAN_DIRECTION]

# while True:
#     # Ask the user for which data they would like to extract
#     column_input = input("Enter the exact column name of the data you would like to extract (EX. TRACK_MEAN_SPEED): ")
#     if column_input in column_titles:
#         break
#     else:
#         print("Invalid entry, please enter a column title exactly as written in column_titles.")

# Create a folder named "trackmateresults" in the selected directory
# trackmateresults_folder = os.path.join(input_directory, "trackmateresults")
# os.makedirs(trackmateresults_folder, exist_ok=True)

# Create a list to store NumPy arrays
result_arrays = []

# Iterate over folders in the directory
for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(folder_path):
        print(f'Processing folder: {folder_name}')
        
        # Check if export.csv exists in the folder
        export_csv_path = os.path.join(folder_path, 'export.csv')
        if not os.path.exists(export_csv_path):
            print(f"export.csv not found in folder '{folder_name}'. Skipping...")
            continue
        
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(export_csv_path)

        # Convert column_input and 'TRACK_DURATION' to numeric values
        for column_input in column_titles:
            df[column_input] = pd.to_numeric(df[column_input], errors='coerce')


        print('original', df.shape)

        # Filter rows starting from the 5th row based on conditions
        filtered_rows = df.iloc[4:].loc[(~df[column_input].isna()) & (df['TRACK_DURATION'] >= track_duration_input)]

        print('filtered', filtered_rows.shape)

        for column in filtered_rows:
            if column in column_titles:
                df_list[column_titles.index(column)] = pd.concat([df_list[column_titles.index(column)], filtered_rows[column]], ignore_index=True)
        
        for df in df_list:
            print(df.head(10))


#         # Extract values from column_input and convert them to a NumPy array
#         result_array = np.array(filtered_rows[column_input])

#         # Add the NumPy array to the list
#         result_arrays.append(result_array)

# if result_arrays:
#     # Create a DataFrame from the list of arrays
#     result_df = pd.DataFrame(result_arrays).T
#     result_df.columns = [f'Column_{i+1}' for i in range(result_df.shape[1])]

#     # Save the resulting DataFrame to a CSV file
#     output_file_path = os.path.join(trackmateresults_folder, f"{column_input}_{track_duration_input}sec.csv")
#     result_df.to_csv(output_file_path, index=False)
#     print("Results saved successfully.")
    
#     # Perform statistical analysis
#     condition_folders = [folder_name for folder_name in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, folder_name))]
    
#     # Calculate mean summary statistics for each condition
#     mean_summaries = [result_df[f'Column_{i+1}'].describe().loc['mean'] for i in range(result_df.shape[1])]

#     # Combine mean summary statistics across conditions
#     combined_summary = pd.concat(mean_summaries, axis=1)

#     # Label combined_summary with condition folder names
#     combined_summary.columns = condition_folders

#     # Perform t-tests between conditions
#     t_test_results = []
#     for i in range(len(condition_folders)):
#         for j in range(i + 1, len(condition_folders)):
#             t_stat, p_value = ttest_ind(combined_summary[condition_folders[i]], combined_summary[condition_folders[j]], nan_policy='omit')
#             significance = 'ns'
#             if p_value < 0.0001:
#                 significance = '****'
#             elif p_value < 0.001:
#                 significance = '***'
#             elif p_value < 0.01:
#                 significance = '**'
#             elif p_value < 0.05:
#                 significance = '*'
#             t_test_results.append((f"T-test between {condition_folders[i]} and {condition_folders[j]} ({column_input}):",
#                                     t_stat, p_value, significance))

#     # Drop rows with missing values
#     clean_combined = combined_summary.dropna()

#     # ANOVA test
#     anova_f_stat, anova_p_value = f_oneway(*[clean_combined[column] for column in clean_combined.columns])
#     anova_significance = 'ns'
#     if anova_p_value < 0.0001:
#         anova_significance = '****'
#     elif anova_p_value < 0.001:
#         anova_significance = '***'
#     elif anova_p_value < 0.01:
#         anova_significance = '**'
#     elif anova_p_value < 0.05:
#         anova_significance = '*'

#     # Create DataFrame to store results
#     statsdf = pd.DataFrame(columns=['Comparison', 'T-Statistic', 'P-Value', 'Significance'])

#     # Populate DataFrame with t-test results
#     for result in t_test_results:
#         statsdf = pd.concat([statsdf, pd.DataFrame([{'Comparison': result[0], 'T-Statistic': result[1], 'P-Value': result[2], 'Significance': result[3]}])], ignore_index=True)

#     # Add empty rows as separation
#     statsdf = pd.concat([statsdf, pd.DataFrame([{'Comparison': None, 'T-Statistic': None, 'P-Value': None, 'Significance': None} for _ in range(2)])], ignore_index=True)

#     # Add ANOVA results
#     statsdf = pd.concat([statsdf, pd.DataFrame([{'Comparison': 'ANOVA', 'T-Statistic': anova_f_stat, 'P-Value': anova_p_value, 'Significance': anova_significance}])],

Processing folder: 1
original (3529, 28)
filtered (383, 28)
       0
0   63.0
1   27.0
2   52.0
3   42.0
4  127.0
5   32.0
6  301.0
7  247.0
8   52.0
9   38.0
     0
0  0.0
1  5.0
2  0.0
3  1.0
4  1.0
5  0.0
6  0.0
7  0.0
8  0.0
9  3.0
     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0
5  0.0
6  0.0
7  0.0
8  0.0
9  0.0
     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0
5  0.0
6  0.0
7  0.0
8  0.0
9  0.0
     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0
5  0.0
6  0.0
7  0.0
8  0.0
9  0.0
     0
0  0.0
1  1.0
2  0.0
3  1.0
4  1.0
5  0.0
6  0.0
7  0.0
8  0.0
9  1.0
          0
0  0.000000
1  4.240051
2  0.000000
3  0.000000
4  0.000000
5  0.000000
6  0.000000
7  0.000000
8  0.000000
9  0.000000
            0
0   65.720787
1   37.100444
2   54.060648
3   44.520533
4  134.621613
5   32.860394
6  318.003809
7  260.763124
8   54.060648
9   42.400508
          0
0  2.937177
1  0.560078
2  0.429168
3  1.331857
4  0.814249
5  1.013558
6  0.530822
7  6.752951
8  0.234415
9  2.026057
            0
0   65.720787
1   32.860

In [7]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create empty DataFrames with column headers
df_list = {title: pd.DataFrame(columns=[title]) for title in column_titles}

# Iterate over folders in the directory
for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(folder_path):
        print(f'Processing folder: {folder_name}')
        
        # Check if export.csv exists in the folder
        export_csv_path = os.path.join(folder_path, 'export.csv')
        if not os.path.exists(export_csv_path):
            print(f"export.csv not found in folder '{folder_name}'. Skipping...")
            continue
        
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(export_csv_path)

        # Convert column_input and 'TRACK_DURATION' to numeric values
        df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

        print('original', df.shape)

        # Filter rows based on conditions
        filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

        print('filtered', filtered_rows.shape)

        # Concatenate filtered rows to the respective DataFrames in df_list
        for column in filtered_rows:
            if column in column_titles:
                df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)

# Print the first 10 rows of each DataFrame
for title, df in df_list.items():
    print(f"DataFrame for {title}:")
    print(df.head(10))


Processing folder: 1
original (3529, 28)
filtered (384, 28)
Processing folder: 2
original (2468, 28)
filtered (513, 28)
Processing folder: 3
original (1534, 28)
filtered (394, 28)
DataFrame for NUMBER_SPOTS:
   NUMBER_SPOTS
0          27.0
1          63.0
2          27.0
3          52.0
4          42.0
5         127.0
6          32.0
7         301.0
8         247.0
9          52.0
DataFrame for NUMBER_GAPS:
   NUMBER_GAPS
0          1.0
1          0.0
2          5.0
3          0.0
4          1.0
5          1.0
6          0.0
7          0.0
8          0.0
9          0.0
DataFrame for NUMBER_SPLITS:
   NUMBER_SPLITS
0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
5            0.0
6            0.0
7            0.0
8            0.0
9            0.0
DataFrame for NUMBER_MERGES:
   NUMBER_MERGES
0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
5            0.0
6            0.0
7            0.0
8            0.0
9        

C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\1088803775.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)
C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\1088803775.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)
C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\1088803775.py:65: FutureWarning: T

In [8]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create empty DataFrames with column headers
df_list = {title: pd.DataFrame(columns=[title]) for title in column_titles}

# Iterate over folders in the directory
for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(folder_path):
        print(f'Processing folder: {folder_name}')
        
        # Check if export.csv exists in the folder
        export_csv_path = os.path.join(folder_path, 'export.csv')
        if not os.path.exists(export_csv_path):
            print(f"export.csv not found in folder '{folder_name}'. Skipping...")
            continue
        
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(export_csv_path)

        # Convert columns to numeric values
        df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

        print('Original DataFrame shape:', df.shape)

        # Count and print the number of empty data points before filtering
        empty_counts_before = df[column_titles].isna().sum()
        print('Empty data points before filtering:')
        print(empty_counts_before)

        # Filter rows based on conditions
        filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

        print('Filtered DataFrame shape:', filtered_rows.shape)

        # Count and print the number of empty data points after filtering
        empty_counts_after = filtered_rows[column_titles].isna().sum()
        print('Empty data points after filtering:')
        print(empty_counts_after)

        # Concatenate filtered rows to the respective DataFrames in df_list
        for column in column_titles:
            if not filtered_rows[column].isna().all():
                df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)

# Print the first 10 rows of each DataFrame
for title, df in df_list.items():
    print(f"DataFrame for {title}:")
    print(df.head(10))


Processing folder: 1
Original DataFrame shape: (3529, 28)
Empty data points before filtering:
NUMBER_SPOTS                           3
NUMBER_GAPS                            3
NUMBER_SPLITS                          3
NUMBER_MERGES                          3
NUMBER_COMPLEX                         3
LONGEST_GAP                            3
TRACK_START                            3
TRACK_STOP                             3
TRACK_DISPLACEMENT                     3
TRACK_DURATION                         3
TRACK_X_LOCATION                       3
TRACK_Y_LOCATION                       3
TRACK_Z_LOCATION                       3
TRACK_MEAN_SPEED                       3
TRACK_MAX_SPEED                        3
TRACK_MIN_SPEED                        3
TRACK_MEDIAN_SPEED                     3
TRACK_STD_SPEED                     1307
TRACK_MEAN_QUALITY                     3
TOTAL_DISTANCE_TRAVELED                3
MAX_DISTANCE_TRAVELED                  3
CONFINEMENT_RATIO                      4
MEAN

C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\920624194.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)
C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\920624194.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_list[column] = pd.concat([df_list[column], filtered_rows[[column]]], ignore_index=True)
C:\Users\oes6098\AppData\Local\Temp\ipykernel_22388\920624194.py:75: FutureWarning: The 

Processing folder: 2
Original DataFrame shape: (2468, 28)
Empty data points before filtering:
NUMBER_SPOTS                          3
NUMBER_GAPS                           3
NUMBER_SPLITS                         3
NUMBER_MERGES                         3
NUMBER_COMPLEX                        3
LONGEST_GAP                           3
TRACK_START                           3
TRACK_STOP                            3
TRACK_DISPLACEMENT                    3
TRACK_DURATION                        3
TRACK_X_LOCATION                      3
TRACK_Y_LOCATION                      3
TRACK_Z_LOCATION                      3
TRACK_MEAN_SPEED                      3
TRACK_MAX_SPEED                       3
TRACK_MIN_SPEED                       3
TRACK_MEDIAN_SPEED                    3
TRACK_STD_SPEED                     534
TRACK_MEAN_QUALITY                    3
TOTAL_DISTANCE_TRAVELED               3
MAX_DISTANCE_TRAVELED                 3
CONFINEMENT_RATIO                     3
MEAN_STRAIGHT_LINE_SPEED  

In [10]:
# working one condition, put data in df

In [9]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create empty DataFrames with column headers
df_list = {title: pd.DataFrame() for title in column_titles}

# Iterate over folders in the directory
for folder_idx, folder_name in enumerate(os.listdir(input_directory)):
    folder_path = os.path.join(input_directory, folder_name)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(folder_path):
        print(f'Processing folder: {folder_name}')
        
        # Check if export.csv exists in the folder
        export_csv_path = os.path.join(folder_path, 'export.csv')
        if not os.path.exists(export_csv_path):
            print(f"export.csv not found in folder '{folder_name}'. Skipping...")
            continue
        
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(export_csv_path)

        # Convert columns to numeric values
        df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

        print('Original DataFrame shape:', df.shape)

        # Count and print the number of empty data points before filtering
        empty_counts_before = df[column_titles].isna().sum()
        print('Empty data points before filtering:')
        print(empty_counts_before)

        # Filter rows based on conditions
        filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

        print('Filtered DataFrame shape:', filtered_rows.shape)

        # Count and print the number of empty data points after filtering
        empty_counts_after = filtered_rows[column_titles].isna().sum()
        print('Empty data points after filtering:')
        print(empty_counts_after)

        # Add filtered rows to the respective DataFrames in df_list as new columns
        for column in column_titles:
            if not filtered_rows[column].isna().all():
                df_list[column][f"{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Print the first 10 rows of each DataFrame
for title, df in df_list.items():
    print(f"DataFrame for {title}:")
    print(df.head(10))


Processing folder: 1
Original DataFrame shape: (3529, 28)
Empty data points before filtering:
NUMBER_SPOTS                           3
NUMBER_GAPS                            3
NUMBER_SPLITS                          3
NUMBER_MERGES                          3
NUMBER_COMPLEX                         3
LONGEST_GAP                            3
TRACK_START                            3
TRACK_STOP                             3
TRACK_DISPLACEMENT                     3
TRACK_DURATION                         3
TRACK_X_LOCATION                       3
TRACK_Y_LOCATION                       3
TRACK_Z_LOCATION                       3
TRACK_MEAN_SPEED                       3
TRACK_MAX_SPEED                        3
TRACK_MIN_SPEED                        3
TRACK_MEDIAN_SPEED                     3
TRACK_STD_SPEED                     1307
TRACK_MEAN_QUALITY                     3
TOTAL_DISTANCE_TRAVELED                3
MAX_DISTANCE_TRAVELED                  3
CONFINEMENT_RATIO                      4
MEAN

In [11]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all condition folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Condition Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each condition
for condition in condition_names:
    condition_path = os.path.join(main_directory, condition)
    
    if not os.path.isdir(condition_path):
        print(f"Condition folder '{condition}' does not exist. Skipping...")
        continue
    
    # Iterate over folders in the condition directory
    for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
        folder_path = os.path.join(condition_path, folder_name)
        
        # Check if the item in the directory is a folder
        if os.path.isdir(folder_path):
            print(f'Processing folder: {folder_name} in condition: {condition}')
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            print('Original DataFrame shape:', df.shape)

            # Count and print the number of empty data points before filtering
            empty_counts_before = df[column_titles].isna().sum()
            print('Empty data points before filtering:')
            print(empty_counts_before)

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            print('Filtered DataFrame shape:', filtered_rows.shape)

            # Count and print the number of empty data points after filtering
            empty_counts_after = filtered_rows[column_titles].isna().sum()
            print('Empty data points after filtering:')
            print(empty_counts_after)

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Print the first 10 rows of each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition}:")
        print(df.head(10))


Processing folder: 0 in condition: wt
Original DataFrame shape: (2330, 28)
Empty data points before filtering:
NUMBER_SPOTS                          3
NUMBER_GAPS                           3
NUMBER_SPLITS                         3
NUMBER_MERGES                         3
NUMBER_COMPLEX                        3
LONGEST_GAP                           3
TRACK_START                           3
TRACK_STOP                            3
TRACK_DISPLACEMENT                    3
TRACK_DURATION                        3
TRACK_X_LOCATION                      3
TRACK_Y_LOCATION                      3
TRACK_Z_LOCATION                      3
TRACK_MEAN_SPEED                      3
TRACK_MAX_SPEED                       3
TRACK_MIN_SPEED                       3
TRACK_MEDIAN_SPEED                    3
TRACK_STD_SPEED                     588
TRACK_MEAN_QUALITY                    3
TOTAL_DISTANCE_TRAVELED               3
MAX_DISTANCE_TRAVELED                 3
CONFINEMENT_RATIO                     3
MEAN_STRA

# Working for collecting data from multiple conditions

In [12]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all condition folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Condition Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each condition
for condition in condition_names:
    condition_path = os.path.join(main_directory, condition)
    
    if not os.path.isdir(condition_path):
        print(f"Condition folder '{condition}' does not exist. Skipping...")
        continue
    
    # Iterate over folders in the condition directory
    for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
        folder_path = os.path.join(condition_path, folder_name)
        
        # Check if the item in the directory is a folder
        if os.path.isdir(folder_path):
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Print the first 10 rows of each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition}:")
        print(df.head(10))


DataFrame for NUMBER_SPOTS in condition wt:
   0_data  1_data  2 bad_data  3_data
0    61.0    52.0        81.0    60.0
1   104.0    80.0        58.0    89.0
2   101.0    78.0        52.0    52.0
3    84.0   166.0        78.0    51.0
4    60.0    70.0        87.0   132.0
5    85.0    50.0        70.0   101.0
6    70.0    61.0        57.0    70.0
7   102.0   301.0        50.0    56.0
8    61.0    49.0        58.0    59.0
9    52.0    90.0        49.0    61.0
DataFrame for NUMBER_GAPS in condition wt:
   0_data  1_data  2 bad_data  3_data
0     0.0     0.0         0.0     1.0
1     1.0     1.0         0.0     0.0
2     0.0     1.0         2.0     0.0
3     3.0     0.0         3.0     0.0
4     4.0     1.0         2.0     0.0
5     1.0     2.0         0.0     0.0
6     4.0     1.0         0.0     0.0
7     1.0     0.0         2.0     0.0
8     0.0     1.0         2.0     1.0
9     1.0     0.0         0.0     1.0
DataFrame for NUMBER_SPLITS in condition wt:
   0_data  1_data  2 bad_data  3

# working 1 replicate, save data

In [3]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all condition folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Condition Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each condition
for condition in condition_names:
    condition_path = os.path.join(main_directory, condition)
    
    if not os.path.isdir(condition_path):
        print(f"Condition folder '{condition}' does not exist. Skipping...")
        continue
    
    # Iterate over folders in the condition directory
    for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
        folder_path = os.path.join(condition_path, folder_name)
        
        # Check if the item in the directory is a folder
        if os.path.isdir(folder_path):
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create trackmate_results folder within each condition folder and save DataFrames
for condition, df_group in df_dict.items():
    condition_results_folder = os.path.join(main_directory, condition, 'trackmate_results')
    
    if not os.path.exists(condition_results_folder):
        os.makedirs(condition_results_folder)
    
    for title, df in df_group.items():
        df.to_csv(os.path.join(condition_results_folder, f"{title}.csv"), index=False)
    
    # Create a text file with the specified message
    with open(os.path.join(condition_results_folder, 'README.txt'), 'w') as file:
        file.write(f'These are the raw metrics extracted and compiled from the TrackMate export.csv files. '
                   f'They are filtered to not include data where the track duration is less than {track_duration_input} seconds.')

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")


DataFrame for NUMBER_SPOTS in condition wt has 4 columns.
DataFrame for NUMBER_GAPS in condition wt has 4 columns.
DataFrame for NUMBER_SPLITS in condition wt has 4 columns.
DataFrame for NUMBER_MERGES in condition wt has 4 columns.
DataFrame for NUMBER_COMPLEX in condition wt has 4 columns.
DataFrame for LONGEST_GAP in condition wt has 4 columns.
DataFrame for TRACK_START in condition wt has 4 columns.
DataFrame for TRACK_STOP in condition wt has 4 columns.
DataFrame for TRACK_DISPLACEMENT in condition wt has 4 columns.
DataFrame for TRACK_DURATION in condition wt has 4 columns.
DataFrame for TRACK_X_LOCATION in condition wt has 4 columns.
DataFrame for TRACK_Y_LOCATION in condition wt has 4 columns.
DataFrame for TRACK_Z_LOCATION in condition wt has 4 columns.
DataFrame for TRACK_MEAN_SPEED in condition wt has 4 columns.
DataFrame for TRACK_MAX_SPEED in condition wt has 4 columns.
DataFrame for TRACK_MIN_SPEED in condition wt has 4 columns.
DataFrame for TRACK_MEDIAN_SPEED in conditi

# Multiple replicate compiling and saving into trackmate results files

In [1]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Create and save DataFrames to the respective condition folders within trackmate_results
for condition, df_group in df_dict.items():
    condition_results_folder = os.path.join(trackmate_results_folder, f"{condition}_compiled_raw")
    
    if not os.path.exists(condition_results_folder):
        os.makedirs(condition_results_folder)
    
    for title, df in df_group.items():
        df.to_csv(os.path.join(condition_results_folder, f"{title}.csv"), index=False)
    
    # Create a text file with the specified message
    with open(os.path.join(condition_results_folder, 'README.txt'), 'w') as file:
        file.write(f'These are the raw metrics extracted and compiled from the TrackMate export.csv files. '
                   f'They are filtered to not include data where the track duration is less than {track_duration_input} seconds.')

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")


Processing replicate: 2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1
  Processing condition: wt
export.csv not found in folder '27del'. Skipping...
export.csv not found in folder '854'. Skipping...
export.csv not found in folder 'wt'. Skipping...
  Processing condition: 27del
  Processing condition: 854
Processing replicate: 2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
DataFrame for NUMBER_SPOTS in condition wt has 14 columns.
DataFrame for NUMBER_GAPS in condition wt has 14 columns.
DataFrame for NUMBER_SPLITS in condition wt has 14 columns.
DataFrame for NUMBER_MERGES in condition wt has 14 columns.
DataFrame for NUMBER_COMPLEX in condition wt has 14 columns.
DataFrame for LONGEST_GAP in condition wt has 14 columns.
DataFrame for TRACK_START in condition wt has 14 columns.
DataFrame for TRACK_STOP in condition wt has 14 columns.
DataFrame for TRACK_DISPLACEMENT in condition wt has 14 columns.
DataFrame

In [3]:
pip install openpyxl

   ---------------------------------------- 0.0/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   --- ----------------------------------- 20.5/251.3 kB 217.9 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/251.3 kB 393.8 kB/s eta 0:00:01
   --------------- ---------------------- 102.4/251.3 kB 653.6 kB/s eta 0:00:01
   -------------------------------- ------- 204.8/251.3 kB 1.0 MB/s eta 0:00:01
   ---------------------------------------  245.8/251.3 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- 251.3/251.3 kB 964.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = 3 #int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = ['wt', '27del', '854'] #[input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = 20 #int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Calculate the mean values for each condition and create a new DataFrame
mean_df_dict = {condition: pd.DataFrame() for condition in condition_names}

for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        if not df.empty:
            mean_value = df.mean(axis=1)
            mean_df_dict[condition][title] = mean_value

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Save the raw DataFrames to Excel files with each metric in a different sheet
for condition, df_group in df_dict.items():
    condition_results_folder = os.path.join(trackmate_results_folder, f"{condition}_compiled_raw")
    
    if not os.path.exists(condition_results_folder):
        os.makedirs(condition_results_folder)
    
    with pd.ExcelWriter(os.path.join(condition_results_folder, f"{condition}_raw_data.xlsx")) as writer:
        for title, df in df_group.items():
            df.to_excel(writer, sheet_name=title, index=False)
    
    # Create a text file with the specified message
    with open(os.path.join(condition_results_folder, 'README.txt'), 'w') as file:
        file.write(f'These are the raw metrics extracted and compiled from the TrackMate export.csv files. '
                   f'They are filtered to not include data where the track duration is less than {track_duration_input} seconds.')

# Save the mean values to an Excel file with each metric in a different sheet
with pd.ExcelWriter(os.path.join(trackmate_results_folder, 'compiled_mean_metrics.xlsx')) as writer:
    for title in column_titles:
        mean_df = pd.DataFrame({condition: df[title] for condition, df in mean_df_dict.items() if title in df.columns})
        mean_df.to_excel(writer, sheet_name=title, index=False)

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")

print(f"Raw data and mean data have been saved to {trackmate_results_folder}")


Processing replicate: 2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: 2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: trackmate_results
Condition folder 'wt' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '27del' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '854' does not exist in replicate 'trackmate_results'. Skipping...


c:\Users\oes6098\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


DataFrame for NUMBER_SPOTS in condition wt has 14 columns.
DataFrame for NUMBER_GAPS in condition wt has 14 columns.
DataFrame for NUMBER_SPLITS in condition wt has 14 columns.
DataFrame for NUMBER_MERGES in condition wt has 14 columns.
DataFrame for NUMBER_COMPLEX in condition wt has 14 columns.
DataFrame for LONGEST_GAP in condition wt has 14 columns.
DataFrame for TRACK_START in condition wt has 14 columns.
DataFrame for TRACK_STOP in condition wt has 14 columns.
DataFrame for TRACK_DISPLACEMENT in condition wt has 14 columns.
DataFrame for TRACK_DURATION in condition wt has 14 columns.
DataFrame for TRACK_X_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_Y_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_Z_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_MEAN_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MAX_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MIN_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MEDIAN_

In [3]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Save the raw DataFrames to Excel files with each metric in a different sheet
with pd.ExcelWriter(os.path.join(trackmate_results_folder, "raw_data.xlsx")) as writer:
    for condition, df_group in df_dict.items():
        for title, df in df_group.items():
            df.to_excel(writer, sheet_name=f"{condition}_{title}", index=False)

# Calculate the mean values for each condition and create a new DataFrame
mean_df_dict = {title: pd.DataFrame(index=condition_names) for title in column_titles}

for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        if not df.empty:
            mean_values = df.mean(axis=1)
            mean_df_dict[title][condition] = mean_values

# Save the mean values to an Excel file with each metric in a different sheet
with pd.ExcelWriter(os.path.join(trackmate_results_folder, 'compiled_mean_metrics.xlsx')) as writer:
    for title, mean_df in mean_df_dict.items():
        mean_df.to_excel(writer, sheet_name=title)

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")

print(f"Raw data and mean data have been saved to {trackmate_results_folder}")


Processing replicate: 2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: 2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854


c:\Users\oes6098\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


DataFrame for NUMBER_SPOTS in condition wt has 14 columns.
DataFrame for NUMBER_GAPS in condition wt has 14 columns.
DataFrame for NUMBER_SPLITS in condition wt has 14 columns.
DataFrame for NUMBER_MERGES in condition wt has 14 columns.
DataFrame for NUMBER_COMPLEX in condition wt has 14 columns.
DataFrame for LONGEST_GAP in condition wt has 14 columns.
DataFrame for TRACK_START in condition wt has 14 columns.
DataFrame for TRACK_STOP in condition wt has 14 columns.
DataFrame for TRACK_DISPLACEMENT in condition wt has 14 columns.
DataFrame for TRACK_DURATION in condition wt has 14 columns.
DataFrame for TRACK_X_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_Y_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_Z_LOCATION in condition wt has 14 columns.
DataFrame for TRACK_MEAN_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MAX_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MIN_SPEED in condition wt has 14 columns.
DataFrame for TRACK_MEDIAN_

c:\Users\oes6098\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [11]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED',
                 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY',
                 'TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
for condition, df_group in df_dict.items():
    with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
        for title, df in df_group.items():
            df.to_excel(writer, sheet_name=title, index=False)

# Calculate the mean values for each condition and create a new DataFrame
mean_df_dict = {title: pd.DataFrame(index=condition_names) for title in column_titles}

# for condition, df_group in df_dict.items():
#     print('condition:', condition)
#     print('df group:', df_group)
#     for title, df in df_group.items():
#         print('title:', title)
#         print('df:', df)
#         if not df.empty:
#             # Calculate the mean for each column
#             mean_values = df.mean(axis=0)
#             mean_df_dict[title][condition] = mean_values
#             print(f"Mean values for {title} in condition {condition}:")
#             print(mean_values)

# Calculate the mean values for each condition and create a new DataFrame
mean_df_dict = {title: pd.DataFrame(index=condition_names) for title in column_titles}

# Iterate over each condition
for condition, df_group in df_dict.items():
    print('Condition:', condition)
    print('DataFrame Group:', df_group)
    
    # Iterate over each metric (title)
    for title, df in df_group.items():
        print('Metric Title:', title)
        print('DataFrame:', df)
        
        # Check if the DataFrame is not empty
        if not df.empty:
            # Calculate the mean for each column in the DataFrame
            mean_values = df.mean(axis=0)
            
            # Assign the mean values to the mean DataFrame for the corresponding metric
            mean_df_dict[title][condition] = mean_values
            
            # Print the mean values for verification
            print(f"Mean values for {title} in condition {condition}:")
            print(mean_values)

# Print the mean DataFrames to check the content
for title, mean_df in mean_df_dict.items():
    print(f"Mean DataFrame for {title}:")
    print(mean_df)

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")


print('test')
print(mean_df_dict)

# Print the mean DataFrames to check the content
for title, mean_df in mean_df_dict.items():
    print(f"Mean DataFrame for {title}:")
    print(mean_df)

# Print the number of columns for each DataFrame for each condition
for condition, df_group in df_dict.items():
    for title, df in df_group.items():
        print(f"DataFrame for {title} in condition {condition} has {df.shape[1]} columns.")

print(f"Raw data has been saved to {trackmate_results_folder}")


Processing replicate: 2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: 2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: trackmate_results
Condition folder 'wt' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '27del' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '854' does not exist in replicate 'trackmate_results'. Skipping...


c:\Users\oes6098\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Condition: wt
DataFrame Group: {'NUMBER_SPOTS':      2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_0_data  \
0                                                 30.0     
1                                                 22.0     
2                                                 21.0     
3                                                 19.0     
4                                                 29.0     
..                                                 ...     
439                                               24.0     
440                                               24.0     
441                                               20.0     
442                                               21.0     
443                                               19.0     

     2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_1_data  \
0                                                 52.0     
1                                                 80.0     
2                                                 2

# need to fix the MEAN

In [12]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
for condition, df_group in df_dict.items():
    with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
        for title, df in df_group.items():
            df.to_excel(writer, sheet_name=title, index=False)



Processing replicate: 2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: 2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3
  Processing condition: wt
  Processing condition: 27del
  Processing condition: 854
Processing replicate: trackmate_results
Condition folder 'wt' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '27del' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '854' does not exist in replicate 'trackmate_results'. Skipping...


c:\Users\oes6098\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Mean values for NUMBER_SPOTS in condition wt:
2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_0_data              41.722973
2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_1_data              44.102151
2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_2 bad_data          35.670823
2.1.24 pvitro 5a, mscarlet lamp, nsc-34 gen1_3_data              46.722973
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_1_data              42.509009
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_10_data             37.045045
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_2_data              46.254505
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_3_data              37.983444
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_4 debatable_data    37.222892
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_5_data              44.501134
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_6_data              38.293930
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_7_data              42.635135
2.8.24 pVitro 5a, mscarlet lamp, nsc-34 gen3_8_data   

In [2]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# # Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
# for condition, df_group in df_dict.items():
#     with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
#         for title, df in df_group.items():
#             df.to_excel(writer, sheet_name=title, index=False)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
for condition, df_group in df_dict.items():
    with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
        for title, df in df_group.items():
            # Truncate the sheet name if it exceeds 31 characters
            sheet_name = title[:31] if len(title) > 31 else title
            df.to_excel(writer, sheet_name=sheet_name, index=False)


# Create a dictionary to hold DataFrames for mean values of each condition
mean_df_dict = {title: pd.DataFrame() for title in column_titles}

# Iterate over each condition
for condition, df_group in df_dict.items():
    print(f"Calculating mean values for condition: {condition}")
    
    # Iterate over each DataFrame in the condition
    for title, df in df_group.items():
        # Calculate mean values for each column
        mean_values = df.mean()
        
        # Append mean values to the corresponding DataFrame in mean_df_dict
        mean_df_dict[title][condition] = mean_values

# Create a folder for mean results
mean_results_folder = os.path.join(main_directory, 'mean_results')
if not os.path.exists(mean_results_folder):
    os.makedirs(mean_results_folder)

# Save the mean DataFrames to separate Excel files for each metric
for title, mean_df in mean_df_dict.items():
    with pd.ExcelWriter(os.path.join(mean_results_folder, f"{title}_mean_values.xlsx")) as writer:
        mean_df.to_excel(writer, index=True)


Processing replicate: 1.12.24 Live pVitro 5A +LAMP
  Processing condition: wt
  Processing condition: 27del


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2865367441.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 854
Processing replicate: 1.17.24 pvitro 5a lamp mscarlet LIVE
  Processing condition: wt


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2865367441.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 27del
  Processing condition: 854
Processing replicate: trackmate_results
Condition folder 'wt' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '27del' does not exist in replicate 'trackmate_results'. Skipping...
Condition folder '854' does not exist in replicate 'trackmate_results'. Skipping...
Calculating mean values for condition: wt
Calculating mean values for condition: 27del
Calculating mean values for condition: 854


In [7]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# # Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
# for condition, df_group in df_dict.items():
#     with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
#         for title, df in df_group.items():
#             df.to_excel(writer, sheet_name=title, index=False)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
for condition, df_group in df_dict.items():
    with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
        for title, df in df_group.items():
            # Truncate the sheet name if it exceeds 31 characters
            sheet_name = title[:31] if len(title) > 31 else title
            df.to_excel(writer, sheet_name=sheet_name, index=False)


# Create a dictionary to hold DataFrames for mean values of each condition
mean_df_dict = {title: pd.DataFrame() for title in column_titles}

# Iterate over each condition
for condition, df_group in df_dict.items():
    print(f"Calculating mean values for condition: {condition}")
    
    # Iterate over each DataFrame in the condition
    for title, df in df_group.items():
        # Calculate mean values for each column, skipping missing values
        mean_values = df.mean(skipna=True)
        
        # Append mean values to the corresponding DataFrame in mean_df_dict
        mean_df_dict[title][condition] = mean_values

# Create a folder for mean results
mean_results_folder = os.path.join(main_directory, 'mean_results')
if not os.path.exists(mean_results_folder):
    os.makedirs(mean_results_folder)

# Save the mean DataFrames to separate Excel files for each metric
for title, mean_df in mean_df_dict.items():
    with pd.ExcelWriter(os.path.join(mean_results_folder, f"{title}_mean_values.xlsx")) as writer:
        mean_df.to_excel(writer, index=True)



Processing replicate: 1.12.24 Live pVitro 5A +LAMP
  Processing condition: wt
  Processing condition: 27del


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\3463133577.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 854
Processing replicate: 1.17.24 pvitro 5a lamp mscarlet LIVE
  Processing condition: wt


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\3463133577.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 27del
  Processing condition: 854
Calculating mean values for condition: wt
Calculating mean values for condition: 27del
Calculating mean values for condition: 854


In [8]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
for condition, df_group in df_dict.items():
    with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
        for title, df in df_group.items():
            # Truncate the sheet name if it exceeds 31 characters
            sheet_name = title[:31] if len(title) > 31 else title
            df.to_excel(writer, sheet_name=sheet_name, index=False)


# Create a dictionary to hold DataFrames for mean values of each condition
mean_df_dict = {title: pd.DataFrame() for title in column_titles}

# Iterate over each condition
for condition, df_group in df_dict.items():
    print(f"Calculating mean values for condition: {condition}")
    
    # Iterate over each DataFrame in the condition
    for title, df in df_group.items():
        print(f"  Processing DataFrame: {title}")
        
        # Calculate mean values for each column, skipping missing values
        mean_values = df.mean(skipna=True)
        print("  Mean values:")
        print(mean_values)
        
        # Append mean values to the corresponding DataFrame in mean_df_dict
        mean_df_dict[title][condition] = mean_values

# Create a folder for mean results
mean_results_folder = os.path.join(main_directory, 'mean_results')
if not os.path.exists(mean_results_folder):
    os.makedirs(mean_results_folder)

# Save the mean DataFrames to separate Excel files for each metric
for title, mean_df in mean_df_dict.items():
    print(f"Saving mean values for metric: {title}")
    with pd.ExcelWriter(os.path.join(mean_results_folder, f"{title}_mean_values.xlsx")) as writer:
        mean_df.to_excel(writer, index=True)

print("Mean value calculation and saving completed.")




Processing replicate: 1.12.24 Live pVitro 5A +LAMP
  Processing condition: wt
  Processing condition: 27del


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2152160861.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 854
Processing replicate: 1.17.24 pvitro 5a lamp mscarlet LIVE
  Processing condition: wt


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2152160861.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 27del
  Processing condition: 854
Calculating mean values for condition: wt
  Processing DataFrame: NUMBER_SPOTS
  Mean values:
1.12.24 Live pVitro 5A +LAMP_10_data           40.765814
1.12.24 Live pVitro 5A +LAMP_11_data           39.081522
1.12.24 Live pVitro 5A +LAMP_2_data            63.967236
1.12.24 Live pVitro 5A +LAMP_3_data            38.585564
1.12.24 Live pVitro 5A +LAMP_4_data            37.260016
1.12.24 Live pVitro 5A +LAMP_5_data            38.009236
1.12.24 Live pVitro 5A +LAMP_6_data            38.972290
1.12.24 Live pVitro 5A +LAMP_7_data            39.415642
1.12.24 Live pVitro 5A +LAMP_8_data            47.370540
1.12.24 Live pVitro 5A +LAMP_9_data            39.066741
1.17.24 pvitro 5a lamp mscarlet LIVE_1_data    57.770884
1.17.24 pvitro 5a lamp mscarlet LIVE_2_data    41.667636
1.17.24 pvitro 5a lamp mscarlet LIVE_3_data    37.142005
1.17.24 pvitro 5a lamp mscarlet LIVE_4_data    38.346984
1.17.24 pvitro 5a lamp mscarlet LIVE_5_data    37.

In [13]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# # Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
# for condition, df_group in df_dict.items():
#     with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
#         for title, df in df_group.items():
#             print('saving', title, 'to', condition)
#             # Truncate the sheet name if it exceeds 31 characters
#             sheet_name = title[:31] if len(title) > 31 else title
#             df.to_excel(writer, sheet_name=sheet_name, index=False)


# Create a dictionary to hold DataFrames for mean values of each condition
mean_df_dict = {}

# Iterate over each metric title
for title in column_titles:
    mean_df_dict[title] = pd.DataFrame()
    print(f"Calculating mean values for metric: {title}")
    
    # Iterate over each condition
    for condition, df_group in df_dict.items():
        print(f"  Calculating mean values for condition: {condition}")
        
        # Get the DataFrame for the current condition and metric
        df = df_group.get(title)
        
        # # Check if the DataFrame exists for the current condition and metric
        # if df is not None:
        #     # Calculate mean values for each column, skipping missing values
        #     mean_values = df.mean(skipna=True)
        #     print(f"    Mean values for condition {condition}:")
        #     print(mean_values)
            
        #     # Append mean values to the corresponding DataFrame in mean_df_dict
        #     mean_df_dict[title][condition] = mean_values
        # else:
        #     print(f"    No data found for condition {condition} and metric {title}")

        # Check if the DataFrame exists for the current condition and metric
        if df is not None:
            # Calculate mean values for each column, skipping missing values
            mean_values = df.mean(skipna=True)
            print(f"    Mean values for condition {condition}:")
            print(mean_values)
            
            # Reset index to ignore the existing row index
            mean_values.reset_index(drop=True, inplace=True)
            
            # Append mean values to the corresponding DataFrame in mean_df_dict
            mean_df_dict[title][condition] = mean_values
        else:
            print(f"    No data found for condition {condition} and metric {title}")


# Create a folder for mean results
mean_results_folder = os.path.join(main_directory, 'mean_results')
if not os.path.exists(mean_results_folder):
    os.makedirs(mean_results_folder)

# Save the mean DataFrames to separate Excel files for each metric
for title, mean_df in mean_df_dict.items():
    print(f"Saving mean values for metric: {title}")
    with pd.ExcelWriter(os.path.join(mean_results_folder, f"{title}_mean_values.xlsx")) as writer:
        mean_df.to_excel(writer, index=True)

print("Mean value calculation and saving completed.")




Processing replicate: 1.12.24 Live pVitro 5A +LAMP
  Processing condition: wt
  Processing condition: 27del


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2984708838.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 854
Processing replicate: 1.17.24 pvitro 5a lamp mscarlet LIVE
  Processing condition: wt


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2984708838.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 27del
  Processing condition: 854
Calculating mean values for metric: NUMBER_SPOTS
  Calculating mean values for condition: wt
    Mean values for condition wt:
1.12.24 Live pVitro 5A +LAMP_10_data           40.765814
1.12.24 Live pVitro 5A +LAMP_11_data           39.081522
1.12.24 Live pVitro 5A +LAMP_2_data            63.967236
1.12.24 Live pVitro 5A +LAMP_3_data            38.585564
1.12.24 Live pVitro 5A +LAMP_4_data            37.260016
1.12.24 Live pVitro 5A +LAMP_5_data            38.009236
1.12.24 Live pVitro 5A +LAMP_6_data            38.972290
1.12.24 Live pVitro 5A +LAMP_7_data            39.415642
1.12.24 Live pVitro 5A +LAMP_8_data            47.370540
1.12.24 Live pVitro 5A +LAMP_9_data            39.066741
1.17.24 pvitro 5a lamp mscarlet LIVE_1_data    57.770884
1.17.24 pvitro 5a lamp mscarlet LIVE_2_data    41.667636
1.17.24 pvitro 5a lamp mscarlet LIVE_3_data    37.142005
1.17.24 pvitro 5a lamp mscarlet LIVE_4_data    38.346984
1.17.24 pvitro 5a

In [14]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
from scipy.stats import ttest_ind, f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the directory containing all replicate folders
main_directory = filedialog.askdirectory(title="Select Main Directory Containing Replicate Folders")

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# Ask the user for the number of conditions
num_conditions = int(input("Enter the number of conditions: "))

# Ask the user for the name of each condition folder
condition_names = [input(f"Enter the name of condition {i+1}: ") for i in range(num_conditions)]

# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 
                 'TRACK_MEAN_SPEED', 'TRACK_MAX_SPEED','TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 'TRACK_STD_SPEED', 
                 'TRACK_MEAN_QUALITY','TOTAL_DISTANCE_TRAVELED', 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 
                 'MEAN_STRAIGHT_LINE_SPEED','LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE']

# Create a dictionary to hold DataFrames for each condition
df_dict = {condition: {title: pd.DataFrame() for title in column_titles} for condition in condition_names}

# Iterate over each replicate folder
for replicate_folder in os.listdir(main_directory):
    replicate_path = os.path.join(main_directory, replicate_folder)
    
    if not os.path.isdir(replicate_path):
        continue
    
    print(f"Processing replicate: {replicate_folder}")
    
    # Iterate over each condition
    for condition in condition_names:
        condition_path = os.path.join(replicate_path, condition)
        
        if not os.path.isdir(condition_path):
            print(f"Condition folder '{condition}' does not exist in replicate '{replicate_folder}'. Skipping...")
            continue
        
        print(f"  Processing condition: {condition}")
        
        # Iterate over folders in the condition directory
        for folder_idx, folder_name in enumerate(os.listdir(condition_path)):
            folder_path = os.path.join(condition_path, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            # Check if export.csv exists in the folder
            export_csv_path = os.path.join(folder_path, 'export.csv')
            if not os.path.exists(export_csv_path):
                print(f"export.csv not found in folder '{folder_name}'. Skipping...")
                continue
            
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(export_csv_path)

            # Convert columns to numeric values
            df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

            # Filter rows based on conditions
            filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

            # Add filtered rows to the respective DataFrames in df_dict as new columns
            for column in column_titles:
                if not filtered_rows[column].isna().all():
                    df_dict[condition][column][f"{replicate_folder}_{folder_name}_data"] = filtered_rows[column].reset_index(drop=True)

# Create the trackmate_results folder in the main directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# # Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
# for condition, df_group in df_dict.items():
#     with pd.ExcelWriter(os.path.join(trackmate_results_folder, f"{condition}_compiled_raw.xlsx")) as writer:
#         for title, df in df_group.items():
#             print('saving', title, 'to', condition)
#             # Truncate the sheet name if it exceeds 31 characters
#             sheet_name = title[:31] if len(title) > 31 else title
#             df.to_excel(writer, sheet_name=sheet_name, index=False)

# Create a dictionary to hold DataFrames for mean values of each condition
mean_df_dict = {title: pd.DataFrame() for title in column_titles}

# Iterate over each metric title
for title in column_titles:
    print(f"Calculating mean values for metric: {title}")
    
    # Iterate over each condition
    for condition, df_group in df_dict.items():
        print(f"  Calculating mean values for condition: {condition}")
        
        # Get the DataFrame for the current condition and metric
        df = df_group.get(title)
        
        # Check if the DataFrame exists for the current condition and metric
        if df is not None:
            # Calculate mean values for each column, skipping missing values
            mean_values = df.mean(skipna=True)
            print(f"    Mean values for condition {condition}:")
            print(mean_values)
            
            # Append mean values to the corresponding DataFrame in mean_df_dict
            # Create a new DataFrame for the current condition if it doesn't exist
            if mean_df_dict[title].empty:
                mean_df_dict[title] = pd.DataFrame(columns=df.columns)
                
            # Concatenate the mean values DataFrame to the existing DataFrame
            mean_df_dict[title] = pd.concat([mean_df_dict[title], mean_values], ignore_index=True)
        else:
            print(f"    No data found for condition {condition} and metric {title}")

# Create a folder for mean results
mean_results_folder = os.path.join(main_directory, 'mean_results')
if not os.path.exists(mean_results_folder):
    os.makedirs(mean_results_folder)

# Save the mean DataFrames to separate Excel files for each metric
for title, mean_df in mean_df_dict.items():
    print(f"Saving mean values for metric: {title}")
    with pd.ExcelWriter(os.path.join(mean_results_folder, f"{title}_mean_values.xlsx")) as writer:
        mean_df.to_excel(writer, index=False)

print("Mean value calculation and saving completed.")




Processing replicate: 1.12.24 Live pVitro 5A +LAMP
  Processing condition: wt
  Processing condition: 27del


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2109075309.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 854
Processing replicate: 1.17.24 pvitro 5a lamp mscarlet LIVE
  Processing condition: wt


C:\Users\oes6098\AppData\Local\Temp\ipykernel_9796\2109075309.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(export_csv_path)


  Processing condition: 27del
  Processing condition: 854
Calculating mean values for metric: NUMBER_SPOTS
  Calculating mean values for condition: wt
    Mean values for condition wt:
1.12.24 Live pVitro 5A +LAMP_10_data           40.765814
1.12.24 Live pVitro 5A +LAMP_11_data           39.081522
1.12.24 Live pVitro 5A +LAMP_2_data            63.967236
1.12.24 Live pVitro 5A +LAMP_3_data            38.585564
1.12.24 Live pVitro 5A +LAMP_4_data            37.260016
1.12.24 Live pVitro 5A +LAMP_5_data            38.009236
1.12.24 Live pVitro 5A +LAMP_6_data            38.972290
1.12.24 Live pVitro 5A +LAMP_7_data            39.415642
1.12.24 Live pVitro 5A +LAMP_8_data            47.370540
1.12.24 Live pVitro 5A +LAMP_9_data            39.066741
1.17.24 pvitro 5a lamp mscarlet LIVE_1_data    57.770884
1.17.24 pvitro 5a lamp mscarlet LIVE_2_data    41.667636
1.17.24 pvitro 5a lamp mscarlet LIVE_3_data    37.142005
1.17.24 pvitro 5a lamp mscarlet LIVE_4_data    38.346984
1.17.24 pvitro 5a

AttributeError: 'DataFrame' object has no attribute 'append'